In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Load your Excel dataset
file_path = "/kaggle/input/nba-thaicomments-dataset/NBA_Comment_Data.xlsx"
df = pd.read_excel(file_path)

# Assuming 'text' column contains the comments and 'label' column contains the labels
df = df.rename(columns={'data': 'text', 'label': 'label'})

# Map labels to integers if necessary
label_map = {'P': 0, 'Neu': 1, 'Neg': 2}
df['label'] = df['label'].map(label_map)

# Convert the DataFrame to a Dataset
dataset = Dataset.from_pandas(df)

# Split the dataset into train, validation, and test sets
dataset_split = dataset.train_test_split(test_size=0.2, seed=42)
test_dataset = dataset_split['test']

# Further split the training set into train and validation sets
train_val_dataset = dataset_split['train'].train_test_split(test_size=0.1, seed=42)  # 10% of train for validation

# Combine the splits into a DatasetDict
dataset_dict = DatasetDict({
    'train': train_val_dataset['train'],
    'validation': train_val_dataset['test'],
    'test': test_dataset
})

# Save the DatasetDict
dataset_dict.save_to_disk('/kaggle/working/nba_comments_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/360 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/40 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("poom-sci/WangchanBERTa-finetuned-sentiment")

tokenizer_config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/305 [00:00<?, ?B/s]

In [3]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [4]:
tokenized = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [5]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

2024-08-05 03:56:31.104329: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 03:56:31.104459: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 03:56:31.236980: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("poom-sci/WangchanBERTa-finetuned-sentiment")

config.json:   0%|          | 0.00/965 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/421M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.136600
1000,0.003700


TrainOutput(global_step=1150, training_loss=0.06113788480344026, metrics={'train_runtime': 153.6195, 'train_samples_per_second': 117.173, 'train_steps_per_second': 7.486, 'total_flos': 425795621679792.0, 'train_loss': 0.06113788480344026, 'epoch': 50.0})

In [8]:
# Save the model and tokenizer
model.save_pretrained("/kaggle/working/saved_model")
tokenizer.save_pretrained("/kaggle/working/saved_model")

('/kaggle/working/saved_model/tokenizer_config.json',
 '/kaggle/working/saved_model/special_tokens_map.json',
 '/kaggle/working/saved_model/sentencepiece.bpe.model',
 '/kaggle/working/saved_model/added_tokens.json',
 '/kaggle/working/saved_model/tokenizer.json')

In [9]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/saved_model")
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/saved_model")


In [10]:
# Load the test dataset
test_dataset = dataset_dict["test"]

# Tokenize the test dataset
def preprocess_test_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length")

tokenized_test = test_dataset.map(preprocess_test_function, batched=True)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [11]:
from transformers import Trainer

# Initialize Trainer with the loaded model
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Evaluate the model
eval_results = trainer.evaluate(eval_dataset=tokenized_test)
print(f"Evaluation results: {eval_results}")


Evaluation results: {'eval_loss': 2.070568561553955, 'eval_runtime': 0.1986, 'eval_samples_per_second': 503.573, 'eval_steps_per_second': 65.464}


In [12]:
from sklearn.metrics import accuracy_score

# Extract predictions
predictions = trainer.predict(tokenized_test)
predicted_labels = predictions.predictions.argmax(axis=-1)
true_labels = tokenized_test["label"]

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.7400
